# Billboard Top 100 to Spotify Playlist

## Introduction

Welcome! In this project, I'll show you how to create a Spotify playlist from the Billboard Hot 100 chart for any date you choose. 🎶

I’m using a combination of web scraping and Spotify API integration:
- **BeautifulSoup** for scraping the Billboard Hot 100.
- **Spotipy** for interacting with the Spotify API to create and manage playlists.

You will need to register with Spotify and get API credentials for this program to work.

Let’s get started!


## Setup and Dependencies

Before diving in, I need to set up the environment and install the required libraries. These include:
- `requests` and `BeautifulSoup` for web scraping.
- `spotipy` for working with Spotify.
- `python-dotenv` to manage environment variables securely.

In [ ]:
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
import os
import requests
import spotipy

## Loading Environment Variables

Now, I'll load environment variables from a `.env` file. This keeps sensitive information like API keys secure and separate from the code.


In [ ]:
# .env plaintext
SPOTIPY_CLIENT_ID=YOUR_SPOTIFY_CLIENT_ID
SPOTIPY_CLIENT_SECRET=YOUR_SPOTIFY_CLIENT_SECRET
SPOTIPY_REDIRECT_URI=http://example.com #Use this example-webpage

In [ ]:
# Load environment variables from .env file
load_dotenv()

## Spotify Authentication

Next, I'll authenticate with Spotify using the Spotipy library. This involves using credentials stored in the `.env` file to access Spotify’s API and manage playlists.

In [ ]:
# Spotify Authentication
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope="playlist-modify-private",
        redirect_uri=os.getenv("SPOTIPY_REDIRECT_URI"),
        client_id=os.getenv("SPOTIPY_CLIENT_ID"),
        client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"),
        show_dialog=True,
        cache_path="token.txt"
    )
)

# Print Spotify User ID
user_id = sp.current_user()["id"]
print(f"Spotify User ID: {user_id}")

## Scraping Billboard Hot 100

I’ll now scrape the Billboard Hot 100 chart for a given date. I’ll fetch the page content and extract the song titles.

In [ ]:
# Scraping Billboard 100
date = input("Which year do you want to travel to? Type the date in this format YYYY-MM-DD: ")
response = requests.get(f"https://www.billboard.com/charts/hot-100/{date}/")
soup = BeautifulSoup(response.text, 'html.parser')

# Extract song names
song_names_spans = soup.select("li ul li h3")
song_names = [song.getText().strip() for song in song_names_spans]
print("Top 100 Songs:")
for index, song in enumerate(song_names[:10], start=1):  # Show the first 10 songs for brevity
    print(f"{index}: {song}")

## Searching for Songs on Spotify

With the song titles in hand, I’ll search for these songs on Spotify and get their URIs. If a song isn’t found, I’ll skip it.

In [ ]:
# Searching Spotify for songs by title
song_uris = []
year = date.split("-")[0]
for song in song_names:
    query = f"track:{song} year:{year}"
    result = sp.search(q=query, type="track")
    print(f"Searching for: {query}")
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song} doesn't exist in Spotify. Skipped.")

print(f"Found {len(song_uris)} songs on Spotify.")

## Creating and Adding to a Spotify Playlist

Finally, I’ll create a new private playlist on Spotify and add the songs I found to this playlist.

In [ ]:
# Creating a new private playlist in Spotify
playlist = sp.user_playlist_create(user=user_id, name=f"{date} Billboard 100", public=False)
print(f"Created Playlist: {playlist['name']} with ID: {playlist['id']}")

# Adding songs found into the new playlist
if song_uris:
    sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)
    print(f"Added {len(song_uris)} songs to the playlist.")
else:
    print("No songs to add to the playlist.")

## Conclusion

Wow, we did it! 🎉

In this project, we turned the Billboard Hot 100 chart into a Spotify playlist. Here’s a quick overview of what we achieved:
1. **Scraped** the top hits from Billboard for any date you’re curious about.
2. **Authenticated** with Spotify, letting us interact with your playlists.
3. **Searched** for each song on Spotify and gathered their URIs.
4. **Created** a shiny new private playlist and **added** all those awesome songs to it!

It's pretty cool to see a historical snapshot of music turned into a personalized playlist, don’t you think? Now you can listen to the top hits from any date as if you were there!

Feel free to play around with this code, maybe add more features like searching for artists or albums, or even share the playlist with friends. 🎶

Thanks for joining me on this musical journey. If you have any questions or suggestions, drop a comment or reach out. Happy listening and have fun with your Spotify playlists! 🚀🎧